In [ ]:
import altair as alt
import numpy as np
from sktime.datasets import load_airline
from sktime.forecasting.naive import NaiveForecaster

In [ ]:
y = load_airline()
data = y.reset_index()
data["Period"] = [i.to_timestamp() for i in data["Period"]]
data["type"] = "historical"

# step 2: specifying forecasting horizon
fh = np.arange(1, 37)
# step 3: specifying the forecasting algorithm
forecaster = NaiveForecaster(strategy="last", sp=12)
# step 4: fitting the forecaster
forecaster.fit(y)
# step 5: querying predictions
y_pred = forecaster.predict(fh)
coverages = [0.95]
y_pred_int = forecaster.predict_interval(coverage=coverages)

In [ ]:
y_pred_int.columns = [x for i in coverages for x in [f"{i}_lower", f"{i}_upper"]]

In [ ]:
data_h = y.reset_index()
data_h["Period"] = [i.to_timestamp() for i in data_h["Period"]]
data_h = data_h.melt(id_vars="Period")

In [ ]:
data = y_pred_int.reset_index()
data["index"] = [i.to_timestamp() for i in data["index"]]
data = data.rename(columns={"index": "Period"})

In [ ]:
y_pred_int

In [ ]:
data = data.melt(id_vars="Period")

In [ ]:
line = (
    alt.Chart(data_h)
    .mark_line(point=True)
    .encode(x="Period", y="value", color="variable")
    .properties(
        width=800,
    )
)

In [ ]:
data

In [ ]:
area = alt.Chart(data).mark_area(opacity=0.3).encode(x="Period", y=alt.Y("value").stack(None), color="variable")

In [ ]:
line + area